In [6]:
from bioservices.uniprot import UniProt
import pandas as pd
import io
u = UniProt(verbose=False)

In [18]:
def search_exact_match(u, gene_name, select_all=False):
    first_query = u.search("{}_HUMAN".format(gene_name), frmt="tab", columns="entry name, id, genes")
    print(first_query == "")
    if first_query == "":
        second_query = u.search("{} AND HUMAN".format(gene_name), frmt="tab", columns="entry name, id, genes")
        df = pd.read_csv(io.StringIO(second_query), sep="\t", dtype="object")
        df[['genes_format']] = df[["Gene names"]].astype(str)
        df['genes_format'] = df['genes_format'].apply(lambda x: x.upper())

        # Querying selecting only HUMAN
        df = df.loc[(df["genes_format"].apply(lambda y: gene_name.upper() in y)) & (df["Entry name"].apply(lambda y: "_HUMAN" in y))]

        # Now we have all entries with _HUMAN and gene in genes_format
        # If select_all return all else return first

        for idx, row in df.iterrows():
            name, uniprot = row["Entry name"], row["Entry"]
            if not select_all:
                return {gene_name: {'gene symbol': name.replace("_HUMAN", ""), 'uniprot ac': uniprot}} 
    else:
        df = pd.read_csv(io.StringIO(first_query), sep="\t", dtype="object")
        for idx, row in df.iterrows():
            name, uniprot = row['Entry name'], row['Entry']
            
        return {gene_name: {'gene symbol': name.replace("_HUMAN", ""), 'uniprot ac': uniprot}}
    return None

search_exact_match(u, "ACNY3")

True


EmptyDataError: No columns to parse from file